In [ ]:
!pip install kafka-python
!pip install tweepy
!pip install pykafka

In [ ]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pykafka

In [ ]:
import tweepy
from tweepy.auth import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

### my twitter cred

In [ ]:
consumer_key = '4eqZwXBNe5V0UZt2y0ompX1OH'
consumer_secret = 'ntUajCkdPgrd9CNm7mmtQ6JP7YwKo00zjktetiqDyQ7lMK1pAp'
access_token = '1418681168258637825-IL5xpVdFdlTwugxS6a6yb3BZpUn0V9'
access_secret = 'Pt34cuHOuaiQrFqFnTlfUY3SBQGfq48k33N8kykY48AJu'

### dont use veer twitter cred


In [ ]:
consumer_key    = 'hzTxnpkRLgCWp8lYDcQYA48Qg'
consumer_secret = 'GXKF11n0QIZSWSyOnRkn1srOuo3HJMMjeIfQHO384Cf9gF5Znh'
access_token    = '135033706-eFg8rNkyEWkAizvhopN9shVw0v8JUO1QAhgCs4HY'
access_secret   = 'ciRuC3KgfLLBvVZWzlD8U6xVMnqfquw4ghuEmIH3sHzNE'

### Ted's creds

In [ ]:
consumer_key = 'ehRQEjyJTzeifkywMW4JLu9HL'
consumer_secret = 'ONzbMlyr0k0Z2FpJdWHbP42vGo74VxvKO3cqbbo8nsKgo0iENG'
access_token = '1417279984302006287-eLoQn9mpx3lYiIJg4qCtmYGoMqazFF'
access_secret = 'aHH71WwKt33DU8ONqgpncA9zry9ndFHtZD35eDH4MUrwL'


## Old Tweet listener, this still works just not well

In [ ]:
class StdOutListener(StreamListener):
    def __init__(self):

        self.client = pykafka.KafkaClient("localhost:9092")
        
        #Get Producer that has topic name is Twitter
        self.producer = self.client.topics[bytes("final_twitter", "ascii")].get_producer()
    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
    def on_data(self, data):
        json_ = json.loads(data) 
        producer.send("basic", json_["text"].encode('utf-8'))
        print(json_["text"])
        return True
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False

    def on_limit(self,status):
        print ("Rate Limit Exceeded, Sleep for 15 Mins")
        time.sleep(15 * 60)
        return True

producer = KafkaProducer(bootstrap_servers='localhost:9092')
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
stream = Stream(auth, l)
stream.filter(track=["covid19", "corona virus"])

## Use this twitter listener

In [ ]:
class TwitterListener(StreamListener):
    def __init__(self):
        self.client = pykafka.KafkaClient("localhost:9092")
        
        #Get Producer that has topic name is Twitter
        self.producer = self.client.topics[bytes("final_twitter", "ascii")].get_producer()
    # overriding the on_data from StreamListner
    def on_connect(self):
        print("Now you have been connected to Twitter Streaming API")
    def on_data(self,data):
        try:
            print(data)
            
            self.producer.produce(bytes(data, "ascii"))
            return True # sending the tweets to producer
        except BaseException as e: 
            print("Error_on_data: %s" % str(e))
            return False
        return True
    def if_error(self,status):
        print(status)
        return True
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False

    def on_limit(self,status_code):
        print ("Rate Limit Exceeded, Sleep for 15 Mins")
        time.sleep(15 * 60)
        return True
 

producer = KafkaProducer(bootstrap_servers=['localhost:9092'])


obj1 = TwitterListener()

#Authorization for Twitter API Access Config
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
    
twitter_stream = Stream(auth, obj1) 
twitter_stream.filter(track=['olympics'], languages = ['en']) # topic that I want to filter    